In [6]:
import numpy as np
import json
from collections import defaultdict
import pandas as pd
import glob
import tqdm
import multiprocessing
import dask.dataframe as dd

In [9]:
def join_weights(rows):
    W = defaultdict(lambda:0, json.loads(rows.at(0)['weights']))
    for row in rows[1:]:
        r = json.loads(row['weights'])
        for k,v in r.items():
            W[k] += v
    return json.dumps(W)
    

def top_fos(w):
    maxv = -1
    maxk = -1
    for k,v in w.items():
        if maxv <= float(v):
            maxv = float(v)
            maxk = k
    return maxk
    
def _step_7(file):
    chunk = dd.read_csv(file, sep='\t', header=None, names=['author_id', 'weights']) # _complete_short_
#     out = open('data/AuthorsFOS_split/authors_weights_year_%d_weights_process_%s' % (YEAR, file.split('_')[-1]), 'w')
    out = open(output1, 'w')
    current = None
    for idx, row in chunk.iterrows():
        current = row['author_id']
        break
        
    weights = defaultdict(lambda:0)
    for idx, row in tqdm.tqdm(chunk.iterrows(), total=len(chunk)):
        if current == row['author_id']:
            w = json.loads(row['weights'])
            for k, v in w.items():
                weights[k] += v
        else:
            if len(weights) > 0:
                f = top_fos(weights)
                if current == 3118234876:
                    print(f, weights)
                out.write("%s\t%s\t%s\n" % (current, json.dumps(weights), f))
            current = row['author_id']
            weights = defaultdict(lambda:0, json.loads(row['weights']))
    
    f = top_fos(weights)
    out.write("%s\t%s\t%s\n" % (current, json.dumps(weights), f))
    out.close()
    del chunk
    
    
def step_7():
    files = glob.glob(source1)
    print(files[:3])
#     from tqdm.contrib.concurrent import process_map
#     process_map(_step_7, files, max_workers=16)
    _step_7(files[0])

In [ ]:
%%time
YEAR = 2020
source1 = 'data/authors_weights_year_%d_sorted.csv' % YEAR
source2 = 'data/AuthorsFOS_split/authors_weights_year_%d_split_sorted_*.csv' % YEAR
output1 = 'data/authors_weights_year_%d_unfied_fos.csv' % YEAR
step_7()